# Your First Click Model: Click Through Rate

This section examines the session data and computes the probability of relevance using Click-Through-Rate. Roughly the number of clicks divided by the number of sessions. Then we examine wheter there's position bias in that data - that is, consider perhaps that some documents have a higher CTR only because they show up higher in the search results.

In [1]:
import sys
sys.path.append("..")
sys.path.append("../ltr")
from aips import *
import matplotlib.pyplot as plt
import numpy
import pandas 
import random

from ltr.sdbn_functions import *
from session_gen import SessionGenerator

# if using a Jupyter notebook, includue:
%matplotlib inline

In [2]:
all_sessions()

,sess_id,query,rank,doc_id,clicked
0,2,ipad,0.0,92636260712,False
1,2,ipad,1.0,635753493559,True
2,2,ipad,2.0,885909393404,False
3,2,ipad,3.0,843404073153,False
4,2,ipad,4.0,885909457595,False
...,...,...,...,...,...
149995,60001,bluray,25.0,23942973416,False
149996,60001,bluray,26.0,25192107191,False
149997,60001,bluray,27.0,27242809710,False
149998,60001,bluray,28.0,600603132872,False


In [3]:
sessions = all_sessions()
products = fetch_products(doc_ids=sessions["doc_id"].unique())

# Listing 11.03

Viewing session 2 of query `transformers dark of the moon` in retrotech. Here we inspect one of the sessions. We encourage you to examine other sessions

In [4]:
query = "transformers dark of the moon"
sessions = get_sessions(query)
sessions[sessions["sess_id"] == 2]

,sess_id,query,rank,doc_id,clicked
0,2,transformers dark of the moon,0.0,47875842328,False
1,2,transformers dark of the moon,1.0,24543701538,True
2,2,transformers dark of the moon,2.0,25192107191,False
3,2,transformers dark of the moon,3.0,47875841420,False
4,2,transformers dark of the moon,4.0,786936817218,False
5,2,transformers dark of the moon,5.0,47875842335,False
6,2,transformers dark of the moon,6.0,47875841406,False
7,2,transformers dark of the moon,7.0,97360810042,False
8,2,transformers dark of the moon,8.0,24543750949,False
9,2,transformers dark of the moon,9.0,36725235564,False


# Listing 11.04

Simple CTR based judgments for our query. We compute the CTR by taking the number of clicks for a document relative to the number of unique sessions the doc appears in for that query.

In [5]:
#%load -s calculate_ctr ../ltr/sdbn_functions.py
def calculate_ctr(sessions):
    click_counts = sessions.groupby("doc_id")["clicked"].sum()
    sess_counts = sessions.groupby("doc_id")["sess_id"].nunique()
    ctrs = click_counts / sess_counts
    return ctrs.sort_values(ascending=False)

In [6]:
query = "transformers dark of the moon"
sessions = get_sessions(query)
ctrs = calculate_ctr(sessions)
ctrs

doc_id
97360810042     0.0824
47875842328     0.0734
47875841420     0.0434
24543701538     0.0364
25192107191     0.0352
786936817218    0.0236
97363560449     0.0192
47875841406     0.0160
400192926087    0.0124
47875842335     0.0106
97363532149     0.0084
36725235564     0.0082
93624956037     0.0082
47875841369     0.0074
24543750949     0.0062
dtype: float64

# Figure 11.2

Source code to render CTR judgment's ideal relevance ranking for `transformers dark of the moon`. In other words, our search results ordered from highest CTR to lowest.

In [7]:
render_judged(products,
              ctrs.to_frame(name="ctr").sort_values("ctr", ascending=False),
              grade_col="ctr",
              label=f"Click-Thru-Rate Judgments for q={query}")

,ctr,image,upc,name,shortDescription
0,0.0824,,97360810042,Transformers: Dark of the Moon - Blu-ray Disc,\N
1,0.0734,,47875842328,Transformers: Dark of the Moon Stealth Force Edition - Nintendo Wii,Transform into an epic hero or a vehicular villain
2,0.0434,,47875841420,Transformers: Dark of the Moon Decepticons - Nintendo DS,Transform into an epic hero or a vehicular villain
3,0.0364,,24543701538,The A-Team - Widescreen Dubbed Subtitle AC3 - Blu-ray Disc,\N
4,0.0352,,25192107191,Fast Five - Widescreen - Blu-ray Disc,\N
5,0.0236,,786936817218,Pirates of the Caribbean: On Stranger Tides - Blu-ray 3D,\N
6,0.0236,,786936817218,Pirates Of The Caribbean: On Stranger Tides (3-D) - Blu-ray 3D,\N
7,0.0192,,97363560449,Transformers: Dark of the Moon - Widescreen Dubbed Subtitle - DVD,\N
8,0.0160,,47875841406,Transformers: Dark of the Moon Autobots - Nintendo DS,Transform into an epic hero or a vehicular villain
9,0.0124,,400192926087,Transformers: Dark of the Moon - Original Soundtrack - CD,\N


# Figure 11.3

Source code to render CTR ideal relevance ranking for `dryer`. Ordering the highest CTR result to the lowest.

In [8]:
sessions = get_sessions("dryer")
ctrs = calculate_ctr(sessions)

render_judged(products,
              ctrs.to_frame(name="ctr").sort_values("ctr", ascending=False),
              grade_col="ctr",
              label=f"Click-Thru-Rate Judgments for q={query}")

,ctr,image,upc,name,shortDescription
0,0.1608,,84691226727,GE - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White,Rotary electromechanical controls; 3 cycles; 3 heat selections; DuraDrum interior; Quiet-By-Design
1,0.0816,,84691226703,Hotpoint - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White-on-White,Rotary controls; 3 cycles; 3 temperature settings; DuraDrum interior
2,0.0710,,12505451713,Frigidaire - Semi-Rigid Dryer Vent Kit - Silver,Expandable vent; custom fitted ends and clamps
3,0.0576,,783722274422,The Independent - Widescreen Subtitle - DVD,\N
4,0.0572,,883049066905,Whirlpool - Affresh Washer Cleaner,Package include 3 tablets; removes and prevents odor-causing residue; compatible with all high-efficiency washers
5,0.0552,,77283045400,Hello Kitty - Hair Dryer - Pink,1875 watts of power; high and low heat settings; cool shot button; detachable styling nozzle
6,0.0546,,74108056764,Conair - Infiniti Ionic Cord-Keeper Hair Dryer - Light Purple,1875 watts; dual voltage; 2 heat and speed settings
7,0.0540,,665331101927,Everything in Static - CD,\N
8,0.0536,,12505525766,Smart Choice - 6' 30 Amp 3-Prong Dryer Cord,Heavy-duty PVC insulation; strain relief safety clamp
9,0.0470,,74108096487,Conair - Infiniti Cord-Keeper Professional Tourmaline Ionic Hair Dryer - Fuchsia,Tourmaline ceramic technology; ionic technology; 1875 watts; Cool Shot function; 3 heat settings; 2 speed settings; 5' retractable cord; includes diffuser


## Listing 11.05

Computing the global CTR of each rank per search ranking to consider whether the click data is biased by position. We look over every search to see what the CTR is when a document is placed in a specific rank.

In [9]:
sessions = get_sessions()
num_sessions = len(sessions["sess_id"].unique())
global_ctrs = sessions.groupby("rank")["clicked"].sum() / num_sessions
global_ctrs

Series([], Name: clicked, dtype: float64)

## Listing 11.06

We look at the documents for our query, and notice that certain ones tend to appear higher and others tend to appear lower. If irrelevant ones dominate the top listings, position bias will dominate our training data

In [10]:
sessions = get_sessions("transformers dark of the moon")
avg_rank = calculate_average_rank(sessions)
avg_rank

doc_id
47875842328      0.9808
24543701538      1.8626
25192107191      2.6596
47875841420      3.5344
786936817218     4.4444
47875842335      5.2776
47875841406      6.1378
97360810042      7.0130
24543750949      7.8626
36725235564      8.6854
47875841369      9.5796
97363560449     10.4304
93624956037     11.3298
97363532149     12.1494
400192926087    13.0526
Name: rank, dtype: float64

In [11]:
render_judged(products, 
              avg_rank.reset_index().sort_values("rank"),
              grade_col="rank",
              label=f"Typical Search Session for q={query}")

,rank,image,upc,name,shortDescription
0,0.9808,,47875842328,Transformers: Dark of the Moon Stealth Force Edition - Nintendo Wii,Transform into an epic hero or a vehicular villain
1,1.8626,,24543701538,The A-Team - Widescreen Dubbed Subtitle AC3 - Blu-ray Disc,\N
2,2.6596,,25192107191,Fast Five - Widescreen - Blu-ray Disc,\N
3,3.5344,,47875841420,Transformers: Dark of the Moon Decepticons - Nintendo DS,Transform into an epic hero or a vehicular villain
4,4.4444,,786936817218,Pirates of the Caribbean: On Stranger Tides - Blu-ray 3D,\N
5,4.4444,,786936817218,Pirates Of The Caribbean: On Stranger Tides (3-D) - Blu-ray 3D,\N
6,5.2776,,47875842335,Transformers: Dark of the Moon Stealth Force Edition - Nintendo 3DS,Transform into an epic hero or a vehicular villain
7,6.1378,,47875841406,Transformers: Dark of the Moon Autobots - Nintendo DS,Transform into an epic hero or a vehicular villain
8,7.0130,,97360810042,Transformers: Dark of the Moon - Blu-ray Disc,\N
9,7.8626,,24543750949,X-Men: First Class - Widescreen Dubbed Subtitle AC3 - Blu-ray Disc,\N


In [12]:
# %load -s calculate_average_rank ../ltr/sdbn_functions.py
def calculate_average_rank(sessions):
    avg_rank = sessions.groupby("doc_id")["rank"].mean()
    return avg_rank.sort_values(ascending=True)

In [13]:
sessions = get_sessions("dryer")
avg_rank = calculate_average_rank(sessions)
avg_rank

doc_id
12505451713      1.9124
84691226727      2.8290
883049066905     3.5726
84691226703      4.4552
74108056764      5.1276
77283045400      5.9318
783722274422     6.7610
665331101927     7.5138
14381196320      8.3308
74108096487      9.1230
36172950027      9.8758
12505525766     10.6500
48231011402     11.4734
74108007469     12.2940
883929085118    12.9996
36725561977     13.8932
36725578241     14.7286
12505527456     15.3526
48231011396     16.1548
856751002097    17.0208
Name: rank, dtype: float64

In [14]:
render_judged(products, 
              avg_rank.reset_index().sort_values("rank"),
              grade_col="rank",
              label=f"Typical Search Session for q={query}")

,rank,image,upc,name,shortDescription
0,1.9124,,12505451713,Frigidaire - Semi-Rigid Dryer Vent Kit - Silver,Expandable vent; custom fitted ends and clamps
1,2.8290,,84691226727,GE - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White,Rotary electromechanical controls; 3 cycles; 3 heat selections; DuraDrum interior; Quiet-By-Design
2,3.5726,,883049066905,Whirlpool - Affresh Washer Cleaner,Package include 3 tablets; removes and prevents odor-causing residue; compatible with all high-efficiency washers
3,4.4552,,84691226703,Hotpoint - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White-on-White,Rotary controls; 3 cycles; 3 temperature settings; DuraDrum interior
4,5.1276,,74108056764,Conair - Infiniti Ionic Cord-Keeper Hair Dryer - Light Purple,1875 watts; dual voltage; 2 heat and speed settings
5,5.9318,,77283045400,Hello Kitty - Hair Dryer - Pink,1875 watts of power; high and low heat settings; cool shot button; detachable styling nozzle
6,6.7610,,783722274422,The Independent - Widescreen Subtitle - DVD,\N
7,7.5138,,665331101927,Everything in Static - CD,\N
8,8.3308,,14381196320,The Mind Snatchers - DVD,\N
9,9.1230,,74108096487,Conair - Infiniti Cord-Keeper Professional Tourmaline Ionic Hair Dryer - Fuchsia,Tourmaline ceramic technology; ionic technology; 1875 watts; Cool Shot function; 3 heat settings; 2 speed settings; 5' retractable cord; includes diffuser


Up next: [Using SDBN Click Model To Overcome Position Bias](2.SDBN-judgments-to-overcome-posn-bias.ipynb)